In [2]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [3]:
#Bridge file that needs correcting
bridges = pd.read_excel("BMMS_overview.xlsx")

In [4]:
bridges.head(10)

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.50,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.40,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.88,12.20,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699833,90.530722,bcs1
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.20,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699667,90.530722,bcs1
5,N1,11.296,Box Culvert,LRP011a,NAYABARI KASPUR BOX CULVERT,8.30,A,101110,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.296,21.45,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.697361,90.533833,bcs1
6,N1,12.239,Box Culvert,LRP012a,KHAS PARA BOX CULVERT,9.30,A,101117,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.239,21.00,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.692333,90.541000,bcs1
7,N1,12.253,Box Culvert,LRP012b,DAWAN BAG BOX CULVERT,6.10,A,101119,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.253,20.60,1987.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.692556,90.541139,bcs1
8,N1,12.660,PC Girder Bridge,LRP013a,Madanpur Bridge.(L),27.50,A,119897,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.66,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Vitikandi,23.685500,90.551278,road_precise
9,N1,12.660,PC Girder Bridge,LRP013a,MADAN PUR (R),26.30,A,109841,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.66,9.20,2003.0,1.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.691111,90.544889,bcs1


<h1>Setting bridges on roads</h1>
There are bridges in places where they shouldn't be with no clear discernible mistake. To fix this the coordinates of all bridges will be adjusted based on the LRP coordinate. The risk of this is that it becomes completely reliant on the new source data, in this case the road file.

In [5]:
#Example of a faulty bridge coordinate. Store it for comparison later
faulty_bridge = bridges[(bridges['road'] == 'Z1811') & ( bridges['LRPName'] == 'LRP115c')]

In [8]:
#road file used for lat/lon
roads = pd.read_csv('Roads_InfoAboutEachLRP.csv')

In [9]:
roads.head(5)

,road,chainage,lrp,lat,lon,type,name
0,N1,0.000,LRPS,23.706028,90.443333,Others,Start of Road after Jatrabari Flyover infront...
1,N1,0.814,LRPSa,23.702917,90.450417,Culvert,Box Culvert
2,N1,0.822,LRPSb,23.702778,90.450472,CrossRoad,Intersection with Z1101
3,N1,1.000,LRP001,23.702139,90.451972,KmPost,Km post missing
4,N1,2.000,LRP002,23.697889,90.460583,KmPost,Km post missing


In [10]:
def bridge_aligner(row):
    """This function takes in a row of the bridges dataframe and then replaces the lat/lon coords using data from the 
        roads file based on their shared LRPs. 
    """
    on_road = bridges.iloc[row.name, 0]
    on_lrp = bridges.iloc[row.name, 3]
    roads_row = roads[(roads['road'] == on_road) & (roads['lrp'] == on_lrp)].index
    if len(roads_row > 0):
        new_lat = roads.iloc[roads_row[0], 3]
        new_lon = roads.iloc[roads_row[0], 4]
        bridges.iloc[row.name, 17] = new_lat
        bridges.iloc[row.name, 18] = new_lon

In [11]:
#Be warned that this takes a long time!
_ = bridges.apply(bridge_aligner, axis=1)

In [12]:
#show old value for comparison
faulty_bridge.iloc[0:, 17:19]

,lat,lon
12252,21.647833,91.3125


In [13]:
#new value
bridges[(bridges['road'] == 'Z1811') & ( bridges['LRPName'] == 'LRP115c')].iloc[0: ,17:19]

,lat,lon
12252,21.644805,92.312611


<h1>Fixing swapped latitudes and longitudes</h1>
There are bridges in wrong positions because their latitudes and longitudes have been swapped around. This section's script focuses on fixing this.

In [14]:
#Create temporary new columns to use in the function
bridges["lon2"] = bridges.loc[:,"lon"]
bridges["lat2"] = bridges.loc[:,"lat"]

In [15]:
def switch_lon(row):
    """This function looks whether any longitude is far out of range of what it could be given the region of interest.
    Based on the outcome a value is stored in the original column.
    """
    if row["lat2"] > 35 and row["lon2"] < 80:
        return row["lat2"]
    else:
        return row["lon2"] 
    
def switch_lat(row):
    """This function looks whether any latitude is far out of range of what it could be given the region of interest.
     Based on the outcome a value is stored in the original column.
    """
    if row["lat2"] > 35 and row["lon2"] < 80:
        return row["lon2"]
    else:
        return row["lat2"]    

In [16]:
#Run the functions to correct for swapped latitudes and longitudes
bridges["lon"] = bridges.apply(switch_lon, axis=1)
bridges["lat"] = bridges.apply(switch_lat, axis=1)

In [17]:
#remove the temporary columns
bridges = bridges.drop("lon2", 1).drop("lat2", 1)

<h1>Removing Duplicate Bridges</h1>
In this section duplicate bridges are removed. Note that there is a difference of information between entries. With more time this information could be gathered into one entry if it is required in later steps.

In [18]:
#Some likely candidates on the N1 road for duplicate removal
bridges[bridges["road"] == "N1"]['LRPName'].value_counts().head(10)

LRP031a    4
LRP023a    3
LRP032d    3
LRP034a    3
LRP018a    3
LRP019a    3
LRP023c    3
LRP029a    3
LRP013a    3
LRP036a    3
Name: LRPName, dtype: int64

In [19]:
#Ignore bridges with zero-value latitudes
bridges = bridges[bridges["lat"] > 0]
#Ignore bridges that don't have a good coordinate approximation
bridges = bridges[bridges["EstimatedLoc"] != "error"]
#Create criteria to use for dropping duplicates
bridges["unique_name"] = bridges["road"].astype("str") + bridges["LRPName"].astype("str") + bridges["chainage"].astype("str")
bridges = bridges.sort_values(by="structureNr")
bridges = bridges.drop_duplicates(subset="unique_name", keep="first")
bridges = bridges.drop("unique_name", 1)
#Reset index after removal for later use
bridges.index = pd.RangeIndex(len(bridges.index))

In [20]:
#Show same counts post removal
bridges[bridges["road"] == "N1"]['LRPName'].value_counts().head(10)

LRP018a    2
LRP280a    1
LRP149b    1
LRP105a    1
LRP370e    1
LRP173a    1
LRP304b    1
LRP416d    1
LRP135a    1
LRP307a    1
Name: LRPName, dtype: int64

<h1>Save updated bridges file</h1>

In [21]:
#Save updated file under a new name. Remember to rename for the java program!
writer = ExcelWriter('BMMS_overview_new.xlsx')
bridges.to_excel(writer,'BMMS_overview',index=False)
writer.save()